In [1]:
import yaml
import lhsmdu
import numpy as np
from scipy.stats import distributions

In [2]:
def make_configs(params, conditions, *, N):
    A = lhsmdu.sample(len(params), N)
    distros = {
        k: distributions.randint(0, len(p_vals))
        for k, p_vals in params.items()
    }
    idx_map = {
        key: np.int32(lhsmdu.inverseTransformSample(distros[key], floats)).flatten().tolist()
        for floats, key in zip(A, params)
    }

    configs = []
    for b in zip(*idx_map.values()):
        d = {k: params[j] for j, (k, params) in zip(b, params.items())}
        if conditions(d):
            configs.append(d)
    return configs

In [3]:
noalias_dumper = yaml.dumper.SafeDumper
noalias_dumper.ignore_aliases = lambda self, data: True
def dump_configs(configs, filename):
    with open(filename, "w") as f:
        f.write(
            "# Copyright (c) 2020, XMOS Ltd, All rights reserved\n"
            "# RANDOMLY GENERATED CONFIGS, MODIFY AT OWN RISK\n"
        )
        yaml.dump({"default": dict(enumerate(configs))}, stream=f, Dumper=noalias_dumper)

# test_conv2d.yml

In [7]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,3,4,5,7,9],
    input_channels=[4,8,12,16,20,32,36,48],
    output_channels=[4,8,12,16,28,32,48,52,64],
    padding=["same", "valid"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [8]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] <= 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["padding"] == "valid":
        if d["K_w"] == d["width"] and d["K_h"] == d["height"]:
            # this would be single pixel conv2d
            return False
        return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]
    return True

In [6]:
configs = make_configs(params, conditions, N=100)
len(configs)

47

In [7]:
dump_configs(configs, "test_conv2d.yml")

# test_custom_relu_conv2d.yml

In [17]:
params["max_value"] = [j/2 for j in range(1, 12)]

In [22]:
configs = make_configs(params, conditions, N=50)
len(configs)

21

In [23]:
dump_configs(configs, "test_custom_relu_conv2d.yml")

# test_padded_conv2d.yml

In [20]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,3,4,5,7,9],
    input_channels=[4,8,12,16,20,32,36,48],
    output_channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [21]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] <= 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["pad_t"] == d["pad_b"] == d["pad_l"] == d["pad_r"] == 0:
        return False
    padded_width = d["width"] + d["pad_l"] + d["pad_r"]
    padded_height = d["height"] + d["pad_t"] + d["pad_b"]
    if d["K_w"] == padded_width and d["K_h"] == padded_height:
        # this would be single pixel conv2d
        return False
    return (
        d["K_w"] <= padded_width
        and d["K_h"] <= padded_height
        and d["pad_r"] < d["K_w"] > d["pad_l"]
        and d["pad_t"] < d["K_h"] > d["pad_b"]
    )    

In [25]:
configs = make_configs(params, conditions, N=70)
len(configs)

20

In [26]:
dump_configs(configs, "test_padded_conv2d.yml")

# test_conv2d_1x1.yml

In [12]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    input_channels=[4,8,12,16,20,32,36,48],
    output_channels=[4,8,12,16,28,32,48,52,64],
    num_threads=[1,2,5],
)

In [13]:
def conditions(d):
    if d["width"] == d["height"] == 1:
        # this would be single pixel conv2d
        return False
    return not (d["num_threads"] > d["height"] or d["num_threads"] > d["width"])

In [14]:
configs = make_configs(params, conditions, N=55)
len(configs)

44

In [15]:
dump_configs(configs, "test_conv2d_1x1.yml")

# test_conv2d_shallowin.yml

In [16]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    input_channels=[4,8,16,32],
    output_channels=[4,8,12,16,28,32,48,52,64],
    padding=["same", "valid"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [17]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] > 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["padding"] == "valid":
        if d["K_w"] == d["width"] and d["K_h"] == d["height"]:
            # this would be single pixel conv2d
            return False
        return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]
    return True

In [18]:
configs = make_configs(params, conditions, N=150)
len(configs)

52

In [19]:
dump_configs(configs, "test_conv2d_shallowin.yml")

# test_padded_conv2d_shallowin.yml

In [93]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    input_channels=[4,8,16,32],
    output_channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [94]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] > 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["pad_t"] == d["pad_b"] == d["pad_l"] == d["pad_r"] == 0:
        return False
    padded_width = d["width"] + d["pad_l"] + d["pad_r"]
    padded_height = d["height"] + d["pad_t"] + d["pad_b"]
    if d["K_w"] == padded_width and d["K_h"] == padded_height:
        # this would be single pixel conv2d
        return False
    return (
        d["K_w"] <= padded_width
        and d["K_h"] <= padded_height
        and d["pad_r"] < d["K_w"] > d["pad_l"]
        and d["pad_t"] < d["K_h"] > d["pad_b"]
    )   

In [96]:
configs = make_configs(params, conditions, N=150)
len(configs)

23

In [97]:
dump_configs(configs, "test_padded_conv2d_shallowin.yml")

# test_depthwise_conv2d.yml

In [24]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    channels=[4,8,12,16,28,32,48,52,64],
    padding=["same", "valid"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [25]:
def conditions(d):
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["padding"] == "valid":
        return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]
    return True

In [26]:
configs = make_configs(params, conditions, N=70)
len(configs)

40

In [27]:
dump_configs(configs, "test_depthwise_conv2d.yml")

# test_padded_depthwise_conv2d.yml

In [85]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [86]:
def conditions(d):
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["pad_t"] == d["pad_b"] == d["pad_l"] == d["pad_r"] == 0:
        return False
    return (
        d["K_w"] <= d["width"] + d["pad_l"] + d["pad_r"]
        and d["K_h"] <= d["height"] + d["pad_t"] + d["pad_b"]
        and d["pad_r"] < d["K_w"] > d["pad_l"]
        and d["pad_t"] < d["K_h"] > d["pad_b"]
    )

In [91]:
configs = make_configs(params, conditions, N=70)
len(configs)

20

In [92]:
dump_configs(configs, "test_padded_depthwise_conv2d.yml")

# test_maxpool2d.yml

In [32]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3],
    K_w=[1,2,3],
    channels=[4,8,12,16,28,32,48,52,64],
    padding=["valid"],
    strides=[(1,1), (1,2), (2,1), (2,2), (1, 3), (3, 2)],
    num_threads=[1,2,5],
)

In [33]:
def conditions(d):
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]

In [43]:
configs = make_configs(params, conditions, N=62)
len(configs)

40

In [44]:
dump_configs(configs, "test_maxpool2d.yml")

# test_avgpool2d.yml

In [52]:
configs = make_configs(params, conditions, N=62)
len(configs)

40

In [53]:
dump_configs(configs, "test_avgpool2d.yml")

# test_global_avgpool2d.yml

In [54]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[4,8,12,16,28,32,48,52,64],
    num_threads=[1,2,5],
)

In [55]:
def conditions(d):
    return not (
        d["num_threads"] > d["height"] or d["num_threads"] > d["width"]
    )

In [64]:
configs = make_configs(params, conditions, N=55)
len(configs)

40

In [65]:
dump_configs(configs, "test_global_avgpool2d.yml")

# Activations

In [71]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[1,4,7,12,16,25,32,49],
)

In [73]:
for file in ["test_relu.yml", "test_relu6.yml", "test_sigmoid.yml", "test_tanh.yml"]:
    configs = make_configs(params, conditions=lambda _: True, N=40)
    print(file, len(configs))
    dump_configs(configs, file)

test_relu.yml 40
test_relu6.yml 40
test_sigmoid.yml 40
test_tanh.yml 40


# test_fully_connected.yml

In [4]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[1,4,7,12,16,25,32,49],
    outputs=[1,4,7,12,16,25,32,49],
    num_threads=[1,2,5],
)

In [5]:
configs = make_configs(params, conditions=lambda _: True, N=40)
len(configs)

40

In [6]:
dump_configs(configs, "test_fully_connected.yml")

# test_single_pixel_conv2d.yml

In [16]:
params = dict(
    K_h=[1,2,4,5,8,12,15],
    K_w=[1,3,4,7,9,14,20],
    input_channels=[1,4,7,12,16,25,32,49],
    output_channels=[1,4,7,12,16,25,32,49],
    num_threads=[1,2,5],
)

In [17]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [18]:
dump_configs(configs, "test_single_pixel_conv2d.yml")